In [36]:
from comet import download_model, load_from_checkpoint
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import os
import requests
from dotenv import load_dotenv

In [2]:
#model_path = download_model("Unbabel/wmt22-comet-da", saving_directory="models/comet_wmt22")
model_path = "models/comet_wmt22/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt"

In [3]:
print(model_path)

models/comet_wmt22/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt


In [4]:
model = load_from_checkpoint(model_path)

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\antoi\OneDrive\Documents\UofT Fall 2024\ESC499 - Thesis\github\translation_thesis\translation_analysis\analysis1\models\comet_wmt22\models--Unbabel--wmt22-comet-da\snapshots\2760a223ac957f30acfb18c8aa649b01cf1d75f2\checkpoints\model.ckpt`
Encoder model frozen.
c:\Users\antoi\OneDrive\Documents\UofT Fall 2024\ESC499 - Thesis\github\translation_thesis\venv\lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [6]:
data = [
    {
        "src": "10 到 15 分钟可以送到吗",
        "mt": "Can I receive my food in 10 to 15 minutes?",
        "ref": "Can it be delivered between 10 to 15 minutes?"
    },
    {
        "src": "Pode ser entregue dentro de 10 a 15 minutos?",
        "mt": "Can you send it for 10 to 15 minutes?",
        "ref": "Can it be delivered between 10 to 15 minutes?"
    }
]
# Call predict method:
model_output = model.predict(data, batch_size=8, gpus=0) # quite rapid

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 1/1 [00:02<00:00,  2.12s/it]


In [7]:
print(model_output)

Prediction([('scores', [0.8417138457298279, 0.7745394110679626]), ('system_score', 0.8081266283988953)])


In [41]:
load_dotenv()  # Load variables from .env file

openai_api_key = os.getenv("OPENAI_API_KEY")
print("API Key loaded:", openai_api_key[:5] + "..." if openai_api_key else "Not found")

# Set your OpenAI API key
client = OpenAI(
    api_key=openai_api_key
)

googletranslate_api_key = os.getenv("GOOGLE_API_KEY")
print("API Key loaded:", googletranslate_api_key[:5] + "..." if googletranslate_api_key else "Not found")
GOOGLE_API_KEY = googletranslate_api_key

API Key loaded: sk-pr...
API Key loaded: AIzaS...


In [45]:
def askChatGPT(user_input, language_input):
    if (language_input == "eng_to_spa"):
        language_selector = "English to Spanish"
    elif (language_input == "spa_to_eng"):
        language_selector = "Spanish to English"
    
    try:
        # Use the OpenAI API to get a response from ChatGPT
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant. Please translate the following text from "+language_selector+". Include the translation and nothing else."
                },
                {
                    "role": "user",
                    "content": user_input
                }
            ]
        )

        #print(completion.choices[0].message)
        result = completion.choices[0].message.content  # Correctly access content
        return result
        # return completion.choices[0]
    
    except Exception as e:
        print(e)
        return "ERROR"

result = askChatGPT("Hola", "spa_to_eng")
print(result)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hello


In [42]:
def askGoogleTranslate(text_to_translate, language_input):
    source_language = ''
    target_language = ''
    if (language_input == "eng_to_spa"):
        source_language = "en"
        target_language = "es"
    elif (language_input == "spa_to_eng"):
        source_language = "es"
        target_language = "en"

    # Make the request to Google Translate API
    try:
        # Set up the URL for Google Translate API
        url = f"https://translation.googleapis.com/language/translate/v2"

        # Payload for the API request
        params = {
            'q': text_to_translate,
            'target': target_language,
            'source': source_language,
            #'key': "invalid_key"
            'key': GOOGLE_API_KEY
        }

        # Make the request to the Google Translate API
        response = requests.get(url, params=params)

        # Check if the response is successful
        if response.status_code != 200:
            print("Google Translate API has returned an error")
            #return jsonify({"error": "Translation API request failed"})
            print(response)
            return response
        

        # Extract the translated text from the API response
        print("translated text returned: "+ str(response.json()))
        #translated_text = response.json().translations[0].translatedText
        translated_text = response.json()['data']['translations'][0]['translatedText']
        print("translated text parsed: "+ translated_text)

        # Return the translated text
        return translated_text

    except Exception as e:
        print(str(e))
        return "ERROR"
    
result = askGoogleTranslate("Hola", "spa_to_eng")
print(result)

translated text returned: {'data': {'translations': [{'translatedText': 'Hello'}]}}
translated text parsed: Hello
Hello


In [11]:
def read_csv_partial_to_numpy(csv_file_path, start_line=100_000, num_lines=100):
    try:
        # Read specific part of CSV: skip first (start_line) rows, then read (num_lines) rows
        df = pd.read_csv(
            csv_file_path,
            encoding='utf-8-sig',
            skiprows=range(1, start_line + 1),  # skip header and up to start_line
            nrows=num_lines  # read only num_lines rows
        )
        
        # Convert DataFrame to NumPy array
        numpy_array = df.to_numpy()
        return numpy_array
    
    except FileNotFoundError:
        print(f"Error: The file '{csv_file_path}' does not exist.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

csv_file_path = '../public_database/data.csv'
numpy_array = read_csv_partial_to_numpy(csv_file_path, start_line=114_800, num_lines=100)

print(numpy_array[:10])

[["Tom told Mary that he wouldn't be doing anything dangerous."
  'Tom le dijo a Mary que él no haría nada peligroso.']
 ['Tom took one of the shirts out of the closet and put it on.'
  'Tom sacó una de las camisas del armario y se la puso.']
 ['Tom took the CD out of its case and put it into the player.'
  'Tom sacó el CD de su caja y lo puso en el reproductor.']
 ['Tom wanted to eat at the small restaurant behind the hotel.'
  'Tom quería comer en el pequeño restaurante detrás del hotel.']
 ['Tom was caught red-handed, stealing from the cash register.'
  'Tom fue atrapado con las manos en la masa, robando de la caja registradora.']
 ['Tom was looking for some people to help him move his piano.'
  'Tom estaba buscando gente que le ayudara a mover su piano.']
 ['Tom was really glad to hear that Mary had passed her exams.'
  'Tom estaba realmente contento al oír que María había aprobado sus exámenes.']
 ['Tom was relieved to hear that Mary had arrived home safely.'
  'Tom estaba aliviad

In [57]:
translated = []
data_array = []
evaluated = []

for i in range(9, 10):
    cur = []
    src = numpy_array[i][0]
    print(src)
    ref = numpy_array[i][1]
    print(ref)
    chatgpt = askChatGPT(src, "eng_to_spa")
    print(chatgpt)
    googletranslate = askGoogleTranslate(src, "eng_to_spa")
    print(googletranslate)

    cur.append(src)
    cur.append(ref)
    cur.append(chatgpt)
    cur.append(googletranslate)
    translated.append(cur)

    #Prepare the data for prediction (src, mt) pairs
    data = [
        {
            "src": src,  # Source sentence
            "ref": ref,
            "mt": chatgpt   # Machine translation
        },
        {
            "src": src,  # Source sentence
            "ref": ref,
            "mt": googletranslate   # Another machine translation for comparison
        }
    ]
    
    # Call the predict method
    model_output = model.predict(data, batch_size=8, gpus=0)
    data_array.append(data)
    evaluated.append(model_output)  # Save the prediction result

translated_array = np.array(translated)
evaluated_array = np.array(evaluated)
data_nparray = np.array(data_array)
np.save('checkpoints/analysis_app_data/translated_114800_100.npy', translated_array)
np.save('checkpoints/analysis_app_data/evaluated_114800_100.npy', evaluated_array)
np.save('checkpoints/analysis_app_data/data_114800_100.npy', data_nparray)
print(evaluated_array[0])
print(translated_array[0])
print(data_nparray[0])

Tom was unable to remember what he had eaten for breakfast.
Tom no podía recordar qué había comido de desayuno.


HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tom no pudo recordar qué había comido para el desayuno.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


translated text returned: {'data': {'translations': [{'translatedText': 'Tom no podía recordar lo que había comido en el desayuno.'}]}}
translated text parsed: Tom no podía recordar lo que había comido en el desayuno.
Tom no podía recordar lo que había comido en el desayuno.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  1.49s/it]

Prediction([('scores', [0.9383423328399658, 0.9229668974876404]), ('system_score', 0.9306546151638031)])
['Tom was unable to remember what he had eaten for breakfast.'
 'Tom no podía recordar qué había comido de desayuno.'
 'Tom no pudo recordar qué había comido para el desayuno.'
 'Tom no podía recordar lo que había comido en el desayuno.']
[{'src': 'Tom was unable to remember what he had eaten for breakfast.', 'ref': 'Tom no podía recordar qué había comido de desayuno.', 'mt': 'Tom no pudo recordar qué había comido para el desayuno.'}
 {'src': 'Tom was unable to remember what he had eaten for breakfast.', 'ref': 'Tom no podía recordar qué había comido de desayuno.', 'mt': 'Tom no podía recordar lo que había comido en el desayuno.'}]
